# Dataset

We will simply try to generate *tiny shakespeare* like text using a **Transofrmer Decoder**

In [1]:
from torch.utils.data import Dataset,DataLoader
from transformers import GPT2Tokenizer

/home/hp/anaconda3/envs/GPT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2") # Use GPT-2 tokenizer (or another tokenizer)
tokenizer.pad_token = tokenizer.eos_token # Add a padding token (if required for batching)

In [ ]:
# __ CONFIG ___

base_model_config = {
    'd_model' : 256,
    'd_k' : 32,
    'd_v' : 32,
    'd_ff' : 1024,
    'n_layers' : 4,
    'n_head' : 8,
    'max_len' : 128,
    'vocab_size' : 10653, #Only the one showing up in shakespear dataset
    'pad_idx':tokenizer.pad_token_id,
    'dropout' : 0.1,
}

mini_model_config = {
    'd_model' : 64,
    'd_k' : 8,
    'd_v' : 8,
    'd_ff' : 256,
    'n_layers' : 2,
    'n_head' : 2,
    'max_len' : 32,
    'vocab_size' : 59514,
    'pad_idx':tokenizer.pad_token_id,
    'dropout' : 0.1,
}

model_config = mini_model_config

train_config = {
    'learning_rate':1e-9,
    'weight_decay':1e-5,
    'checkpoint_dir':".",
    'num_epochs': 50,
    'warmup_steps': 1000,
    'batch_size':64
}

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer

#Custom Dataset
class TextDataset(Dataset):
    def __init__(self, text, tokenizer, seq_length):
        """
        Args:
            text (str): Raw text data.
            tokenizer: Tokenizer to convert text to tokens.
            seq_length (int): Length of each sequence.
        """
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        
        # Tokenize the text and convert to input IDs
        self.tokens = tokenizer.encode(text,return_tensors="pt",add_special_tokens=False)[0]
        
        # Chunk tokens into sequences of fixed length
        self.num_sequences = self.tokens.size(-1) // seq_length
        self.tokens = self.tokens[: self.num_sequences * seq_length]  # Trim extra tokens

    def __len__(self):
        # Number of sequences
        return self.num_sequences

    def __getitem__(self, idx):
        # Return a sequence of fixed length
        start_idx = idx * self.seq_length
        end_idx = start_idx + self.seq_length
        sequence = torch.cat((torch.tensor((self.tokenizer.bos_token_id,)),self.tokens[start_idx:end_idx]),dim=-1)
        sequence = torch.tensor(sequence, dtype=torch.long)
        return sequence[:-1],sequence[1:]

In [5]:
# Path to the .txt file
file_path = "tiny_shakespeare.txt"

# Open the file and read its contents as a single string
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

In [ ]:
#Train/Val split
text_lengh = len(text)
sep = int(text_lengh*train_config['train_per'])
train_text = text[:sep]
val_text = text[sep:]

#Create a dataset
train_dataset = TextDataset(train_text, tokenizer, model_config['max_len'])
val_dataset = TextDataset(val_text, tokenizer, model_config['max_len'])

# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=train_config['batch_size'], shuffle=True,drop_last=True)
val_loader = DataLoader(train_dataset, batch_size=train_config['batch_size'], shuffle=True,drop_last=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (34220 > 1024). Running this sequence through the model will result in indexing errors


## Model

In [7]:
from transformer.model import TransformerDecoder

In [8]:
model = TransformerDecoder(**model_config)

## Trainer

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR
from torch.amp import autocast, GradScaler #Package for mixed precision in cuda computations

from typing import Dict
from pathlib import Path
import logging
from sacrebleu.metrics import BLEU, CHRF, TER
from tqdm import tqdm
import wandb #Visualisations

In [10]:
d = torch.device("cuda" if torch.cuda.is_available() else "cpu")
d.type

'cpu'

In [ ]:
class CustomLRScheduler:
    def __init__(self, optimizer, d_model, warmup_steps):
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.step_num = 0

    def step(self):
        self.step_num += 1
        lrate = (self.d_model ** -0.5) * min(
            self.step_num ** -0.5, 
            self.step_num * (self.warmup_steps ** -1.5)     
        )
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lrate

class Trainer:
    def __init__(self,model:nn.Module,train_loader:DataLoader,val_loader:DataLoader,tokenizer:GPT2Tokenizer,config:dict):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.tokenizer = tokenizer
        self.config = config

        # Setup device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Initialize optimizer
        self.optimizer = Adam(
            self.model.parameters(),
            lr=config['learning_rate'],
            weight_decay=config['weight_decay'],
            betas=(0.9,0.98)
        )

        # # Define the learning rate scheduler
        self.scheduler = CustomLRScheduler(self.optimizer,model_config['d_model'],train_config['warmup_steps'])

        #Loss
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=model.pad_idx) 

        #Mixed precision
        self.scaler = GradScaler()

        #Create checkpoint directory
        self.checkpoint_dir = Path(config['checkpoint_dir'])
        self.checkpoint_dir.mkdir(exist_ok=True)

        # Initialize logging ???
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def save_checkpoint(self, epoch: int, metrics: Dict[str, float]):
        """Save model checkpoint"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'metrics': metrics
        }
        
        path = self.checkpoint_dir / f"checkpoint_epoch_{epoch}.pt"
        torch.save(checkpoint, path)
        self.logger.info(f"Saved checkpoint to {path}")

    def train_epoch(self,epoch:int):
        """
        Train for one epoch
        """
        self.model.train() #Set to train
        total_loss = 0
        progress_bar = tqdm(self.train_loader,desc=f'Epoch {epoch}')

        for batch_idx,batch in enumerate(progress_bar):
            #Move batch to device
            src_seq,tgt_seq = batch
            src_seq,tgt_seq = src_seq.to(self.device),tgt_seq.to(self.device)

            #Mixed precision training
            with autocast(self.device.type):
                outputs = self.model(src_seq,None) #logits of shape (batch_size,max_len,vocab_size)
                loss = self.loss_fn(outputs.view(train_config['batch_size']*model_config['max_len'],-1).softmax(dim=-1),tgt_seq.flatten())

            #Scale loss and compute gadients
            self.scaler.scale(loss).backward()

            #Gradient clipping
            self.scaler.unscale_(self.optimizer)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(),1.0)

            #Update weights
            self.scaler.step(self.optimizer)
            self.scaler.update()
            self.scheduler.step()

            #Zero gradients
            self.optimizer.zero_grad()

            #update total loss
            total_loss+=loss.item()

            # Update progress bar
            progress_bar.set_postfix({
                'loss': f"{total_loss / (batch_idx + 1):.4f}",
                'lr': f"{self.scheduler.optimizer.param_groups[0]['lr']:.2e}"
            })

            #Log to wandb
            if wandb.run is not None:
                wandb.log({
                    'train_loss':loss.item(),
                    'learning_rate':self.scheduler.optimizer.param_groups[0]['lr'],
                })

        # Compute epoch averages
        num_batches = len(self.train_loader)
        epoch_loss = total_loss / num_batches

        return epoch_loss,None
    
    def train(self):
        """Full training loop"""
        for epoch in range(train_config['num_epochs']):
            self.logger.info(f"\nStarting epoch {epoch + 1}")
            
            # Training phase
            train_loss, train_metrics = self.train_epoch(epoch)
            
            # Validation phase
            #val_loss, val_metrics = self.validate()
            
            # Log metrics
            metrics = {
                'epoch': epoch + 1,
                'train_loss': train_loss,
                #'val_loss': val_loss,
                #**{f"train_{k}": v for k, v in train_metrics.items()},
                #**{f"val_{k}": v for k, v in val_metrics.items()}
            }
            
            self.logger.info(
                f"Epoch {epoch + 1} - "
                f"Train loss: {train_loss:.4f} - "
                #f"Val loss: {val_loss:.4f} - "
               # f"Val BLEU: {val_metrics['bleu']:.2f}"
            )
            
            if wandb.run is not None:
                wandb.log(metrics)
            
            # Save checkpoint if best so far
            #if val_metrics['bleu'] > best_bleu:
                #best_bleu = val_metrics['bleu']
                #self.save_checkpoint(epoch, metrics)


In [12]:
trainer = Trainer(model,train_loader,val_loader,tokenizer,train_config)

/home/hp/anaconda3/envs/GPT/lib/python3.12/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [13]:
trainer.train()

INFO:__main__:
Starting epoch 1
Epoch 0:   0%|          | 0/16 [00:00<?, ?it/s]/tmp/ipykernel_4555/2966730660.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence = torch.tensor(sequence, dtype=torch.long)
Epoch 0: 100%|██████████| 16/16 [00:12<00:00,  1.26it/s, loss=10.9940, lr=6.32e-05]
INFO:__main__:Epoch 1 - Train loss: 10.9940 - 
INFO:__main__:
Starting epoch 2
Epoch 1: 100%|██████████| 16/16 [00:11<00:00,  1.35it/s, loss=10.9940, lr=1.26e-04]
INFO:__main__:Epoch 2 - Train loss: 10.9940 - 
INFO:__main__:
Starting epoch 3
Epoch 2:  50%|█████     | 8/16 [00:06<00:06,  1.19it/s, loss=10.9940, lr=1.58e-04]


KeyboardInterrupt: 

In [ ]:
loss_fn = nn.CrossEntropyLoss()

for epoch in range(50):
    progress_bar = tqdm(val_loader,desc=f'Epoch {epoch}')
    total_loss = 0
    for batch_idx,batch in enumerate(progress_bar):
        #Move batch to device
        src_seq,tgt_seq = batch
        src_seq,tgt_seq = src_seq.to(d),tgt_seq.to(d)
        print(src_seq.shape,tgt_seq.shape)

        #Mixed precision training
        with autocast(d.type):
            outputs = model(src_seq,None) #logits of shape (batch_size,max_len,vocab_size)
            loss = loss_fn(outputs.view(train_config['batch_size']*model_config['max_len'],-1).softmax(dim=-1),tgt_seq.flatten())

        #update total loss
        total_loss+=loss.item()

        # Update progress bar
        progress_bar.set_postfix({
            'loss': f"{total_loss / (batch_idx + 1):.4f}"
        })




In [ ]:
for batch in train_loader:
    print(batch.shape)
    break

torch.Size([64, 32])


/tmp/ipykernel_250165/2304055213.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sequence, dtype=torch.long)
